# Notes

code for producing GOIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [219]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [220]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

In [221]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

## import data

In [240]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel==fuel_type]

owners_df_orig = owners_df_orig.loc[owners_df_orig['ProjectID']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']

owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A3')
parent_metadata_df.set_index('Parent', inplace=True)

In [241]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/1702877721.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  country_ratios_df.replace('--', numpy.nan, inplace=True)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/1702877721.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  owners_df_orig.replace('--',numpy.nan,inplace=True)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/1702877721.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a f

In [242]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
#region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [243]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([(  'Africa',                 'Northern Africa'),
            (  'Africa',              'Sub-Saharan Africa'),
            ('Americas', 'Latin America and the Caribbean'),
            ('Americas',                'Northern America'),
            (    'Asia',                    'Central Asia'),
            (    'Asia',                    'Eastern Asia'),
            (    'Asia',              'South-eastern Asia'),
            (    'Asia',                   'Southern Asia'),
            (    'Asia',                    'Western Asia'),
            (  'Europe',                  'Eastern Europe'),
            (  'Europe',                 'Northern Europe'),
            (  'Europe',                 'Southern Europe'),
            (  'Europe',                  'Western Europe'),
            ( 'Oceania',       'Australia and New Zealand'),
            ( 'Oceania',                       'Melanesia'),
            ( 'Oceania',                      'Micronesia'),
            ( 'Oceania',

## file names with regional specifics

In [244]:
if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-gas-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-NGL-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-Oil-pipelines-'+str(datetime.date.today())+'.xlsx')

### create country-specific dataframes for region, country_ratios_df, owners_df

In [245]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

# owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
#                                             '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/3574110634.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/3574110634.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(


In [246]:
country_ratios_df

,PipelineName,SegmentName,ProjectID,Country,LengthEstimateKmByCountry,LengthPerCountryFraction,Region,SubRegion,RegionOld,PipelineBubbleRegion,...,Parent,H2Status,H2Type,CancelledYear,ProposalYear,ConstructionYear,ShelvedYear,StartYearEarliest,StartCountry,EndCountry
0,Alberta Clipper Oil Pipeline,,P0001,Canada,1066.328784,0.680000,Americas,Northern America,North America,North America,...,Enbridge [100.00%],NaN,NaN,,,,,2010,Canada,United States
1,Alberta Clipper Oil Pipeline,,P0001,United States,512.042577,0.320000,Americas,Northern America,North America,North America,...,Enbridge [100.00%],NaN,NaN,,,,,2010,Canada,United States
2,Athabasca Oil Pipeline,,P0002,Canada,522.239984,1.000000,Americas,Northern America,North America,North America,...,Enbridge [88.43%]; 23 Indigenous communities [...,NaN,NaN,,,,,1999,Canada,Canada
3,Bakken Expansion Pipeline,,P0004,Canada,150.276903,0.595268,Americas,Northern America,North America,North America,...,Enbridge [100.00%],NaN,NaN,,,,,2013,United States,Canada
4,Bakken Expansion Pipeline,,P0004,United States,102.1756,0.400000,Americas,Northern America,North America,North America,...,Enbridge [100.00%],NaN,NaN,,,,,2013,United States,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6585,Nigeria–Libya Gas Pipeline,,P7102,Niger,1241.350811,0.389414,Africa,Sub-Saharan Africa,Sub-Saharan Africa,Africa,...,NaN,NaN,,,2024,,,NaN,Nigeria,Libya
6586,Pomeranian Hydrogen Cluster,SYSTEM/NETWORK INFO,P7103,Poland,561.549529,0.999882,Europe,Eastern Europe,Europe,Europe,...,NaN,NaN,new AND converted segments,,,,,2029,Poland,Poland
6587,Pomeranian Hydrogen Cluster,SYSTEM/NETWORK INFO,P7103,Germany,0.066229,0.000118,Europe,Western Europe,Europe,Europe,...,NaN,NaN,new AND converted segments,,,,,2029,Poland,Poland
6592,Malaysia Singapore Gas Pipeline,,P7105,Malaysia,35,0.500000,Asia,South-eastern Asia,SE Asia,Asia Pacific,...,NaN,NaN,,,,,,1991,Malaysia,Singapore


In [247]:
pipes_df_touse.head()

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Status,Status [ref],Researcher,LastUpdated,Fuel,...,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner [ref],ImpactedByRussiaUkraineInvasion,EGTImport,ChinesePipelineType,ChineseNetworkPrimary,ChineseNetworkSecondary,ChineseClassificationNotes
0,,Double E Pipeline Project,,https://www.gem.wiki/Double_E_Pipeline_Project,P0061,operating,,HK,2023-08-21,Gas,...,,,,,,,,,,
1,,Acadian Gas Pipeline System,,https://www.gem.wiki/Acadian_Gas_Pipeline_System,P0143,operating,,HK,2023-08-18,Gas,...,,,,,,,,,,
2,,Algonquin Gas Transmission Pipeline,Access Northeast Gas Pipeline,https://www.gem.wiki/Algonquin_Gas_Transmissio...,P0144,cancelled,,HK,2023-08-21,Gas,...,,,,,,,,,,
3,,Access South Gas Pipeline,,https://www.gem.wiki/Access_South_Gas_Pipeline,P0145,operating,,HK,2023-08-21,Gas,...,,,,,,,,,,
4,,Alaska Gas Pipeline,,https://www.gem.wiki/Alaska_Gas_Pipeline,P0146,cancelled,,WA,2023-09-06,Gas,...,,,,,,,,,,


### sum LengthMergedKmByCountry and MergedKmByRegion

In [248]:
status_list = ['proposed', 
               'construction', 
               'shelved', 
               'cancelled', 
               'operating', 
               'idle', 
               'mothballed', 
               'retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [249]:
excel_status_list = ['proposed', 
                     'construction', 
                     'in development (proposed + construction)', 
                     'shelved', 
                     'cancelled', 
                     'operating', 
                     'idle', 
                     'mothballed', 
                     'retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [250]:
country_ratios_df_subset_status

,PipelineName,SegmentName,ProjectID,Country,LengthEstimateKmByCountry,LengthPerCountryFraction,Region,SubRegion,RegionOld,PipelineBubbleRegion,...,Parent,H2Status,H2Type,CancelledYear,ProposalYear,ConstructionYear,ShelvedYear,StartYearEarliest,StartCountry,EndCountry
563,Saleh–Ras al-Khaimah gas Pipeline,,P0522,United Arab Emirates,39.283529,1.000000,Asia,Western Asia,Middle East and North Africa,Middle East,...,RAK Gas [100.00%],NaN,,,,,,NaN,United Arab Emirates,United Arab Emirates
991,Stavropol-Moscow Gas Pipeline,,P0793,Ukraine,227.711974,0.210000,Europe,Eastern Europe,Europe,Europe,...,Gazprom [100.00%],NaN,,,,,,NaN,Russia,Russia
992,Stavropol-Moscow Gas Pipeline,,P0793,Russia,882.736527,0.794937,Europe,Eastern Europe,Eurasia,Eurasia,...,Gazprom [100.00%],NaN,,,,,,NaN,Russia,Russia
1084,Sangu Gas Transmission Pipeline,,P0876,Bangladesh,43.476366,1.000000,Asia,Southern Asia,South Asia,Asia Pacific,...,Santos Limited [100.00%],NaN,,,,,,1996,Bangladesh,Bangladesh
1591,Taganrog-Mariupol-Berdyansk Gas Pipeline,,P1457,Ukraine,118.269729,0.700000,Europe,Eastern Europe,Europe,Europe,...,Gazprom [unknown %]; JSC Mahistralni Gazoprovo...,NaN,,,,,,NaN,Russia,Ukraine
1592,Taganrog-Mariupol-Berdyansk Gas Pipeline,,P1457,Russia,51.896838,0.300000,Europe,Eastern Europe,Eurasia,Eurasia,...,Gazprom [unknown %]; JSC Mahistralni Gazoprovo...,NaN,,,,,,NaN,Russia,Ukraine
1658,Iwaki Offshore line,,P1548,Japan,40.061586,1.000000,Asia,Eastern Asia,East Asia,Asia Pacific,...,INPEX Corporation [100.00%],NaN,,,,,,1983,Japan,Japan
1702,Llandarcy-Bacton Gas Pipeline,,P1599,United Kingdom,402.690013,1.000000,Europe,Northern Europe,Europe,Europe,...,Independent Oil & Gas PLC [100.00%],NaN,,,,,,NaN,United Kingdom,United Kingdom
1718,Messoyakha-Norilsk Pipeline,I,P1647,Russia,231.372775,1.000000,Europe,Eastern Europe,Eurasia,Eurasia,...,Nornickel PJSC [100.00%],NaN,,,,1967,,1969,Russia,Russia
1726,Goldeneye Field–St. Fergus Pipeline,,P1681,United Kingdom,101.911891,1.000000,Europe,Northern Europe,Europe,Europe,...,Shell [100.00%],NaN,,,,,,NaN,United Kingdom,United Kingdom


In [251]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['LengthMergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['LengthMergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['in development (proposed + construction)'] = km_by_region[['proposed','construction']].sum(axis=1)
km_by_country['in development (proposed + construction)'] = km_by_country[['proposed','construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired
===regional calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/2198785467.py:40: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_region.to_excel(excel_writer, 'Kilometers by region')
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/2198785467.py:41: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_country.to_excel(excel_writer, 'Kilometers by country')


In [252]:
km_by_region

proposed construction  \
Region   Subregion                                                 
Africa   Northern Africa                     6139.5      1121.55   
         Sub-Saharan Africa                 19388.3       1582.0   
Americas Latin America and the Caribbean   15555.28       3008.0   
         Northern America                   7980.47      3224.26   
Asia     Central Asia                       4554.54       813.77   
         Eastern Asia                      42160.83     27973.35   
         South-eastern Asia                 8849.03        666.0   
         Southern Asia                     10651.84     23337.86   
         Western Asia                       4483.75      9772.55   
Europe   Eastern Europe                    22157.16      5053.19   
         Northern Europe                     972.49          1.2   
         Southern Europe                    6765.84        986.0   
         Western Europe                      905.55         98.0   
Oceania  Australia and New Zealand          9856.64          4.3   
         Melanesia                            320.0                
         Micronesia                                                
         Polynesia                                                 
Total                                     160741.22     77642.03   

                                         in development (proposed + construction)  \
Region   Subregion                                                                  
Africa   Northern Africa                                                  7261.05   
         Sub-Saharan Africa                                               20970.3   
Americas Latin America and the Caribbean                                 18563.28   
         Northern America                                                11204.73   
Asia     Central Asia                                                     5368.31   
         Eastern Asia                                                    70134.18   
         South-eastern Asia                                               9515.03   
         Southern Asia                                                    33989.7   
         Western Asia                                                     14256.3   
Europe   Eastern Europe                                                  27210.35   
         Northern Europe                                                   973.69   
         Southern Europe                                                  7751.84   
         Western Europe                                                   1003.55   
Oceania  Australia and New Zealand                                        9860.94   
         Melanesia                                                          320.0   
         Micronesia                                                                 
         Polynesia                                                                  
Total                                                                   238383.25   

                                           shelved cancelled   operating  \
Region   Subregion                                                         
Africa   Northern Africa                    412.39     275.0    42476.47   
         Sub-Saharan Africa                1274.72    176.96     7141.52   
Americas Latin America and the Caribbean   4734.05   3562.41    61912.79   
         Northern America                  3818.47  20654.99   400989.45   
Asia     Central Asia                       966.33   1990.59    32487.69   
         Eastern Asia                      7285.12   4424.02    144942.7   
         South-eastern Asia                2562.41    1479.0    17603.13   
         Southern Asia                      2199.0  12202.97    40327.49   
         Western Asia                      2760.69   9003.75    37570.31   
Europe   Eastern Europe                    2450.92   9887.87   145167.38   
         Northern Europe                    1000.0   1003.77    25671

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

### pipeline km by start year, type

In [235]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel']=='Gas')]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel']=='Oil')]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel']=='NGL')]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [236]:
pipes_started_sum

StartYearEarliest
1904.0     9334.00
1920.0     5413.83
1927.0     7491.50
1928.0    12231.00
1929.0     1384.04
            ...   
2023.0     5255.65
2024.0     3180.64
2025.0       19.50
2026.0      188.00
2030.0      100.00
Name: LengthMergedKm, Length: 89, dtype: float64

In [237]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['Oil pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_32072/652397138.py:21: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')


## save excel file

In [238]:
excel_writer.close()

## calculating stats for landing page

In [196]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type]['LengthMergedKm'].sum()/1e6, 'M km tracked')

3860 gas pipeline projects tracked
1.42087998 M km tracked


In [197]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['proposed','construction']))].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['proposed','construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

862 gas pipeline projects tracked
238.31112000000002 K km tracked
